# Setup

In [ ]:
%cd /kaggle/working/
%rm -rf mlproduction
!git clone https://github.com/daoanhkhoa123/mlproduction.git
%cd /kaggle/working/mlproduction
!ls

/kaggle/working
Cloning into 'mlproduction'...
remote: Enumerating objects: 409, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 409 (delta 48), reused 67 (delta 27), pack-reused 317 (from 1)
Receiving objects: 100% (409/409), 3.60 MiB | 18.60 MiB/s, done.
Resolving deltas: 100% (236/236), done.
/kaggle/working/mlproduction
main.py  pyproject.toml  README.md  runtime  scripts  sqls  src  uv.lock


In [ ]:
!chmod +x scripts/*.sh
!scripts/cloud_setup.sh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 65.2 MB/s eta 0:00:00:00:0100:01
Resolved 69 packages in 711ms                                        
# This file was autogenerated by uv via the following command:
#    uv pip compile pyproject.toml -o requirements.txt --extra-index-url https://pypi.org/simple --emit-index-url
--index-url https://pypi.org/simple
--extra-index-url https://pypi.org/simple

aiohappyeyeballs==2.6.1
    # via aiohttp
aiohttp==3.13.2
    # via fsspec
aiosignal==1.4.0
    # via aiohttp
annotated-types==0.7.0
    # via pydantic
attrs==25.4.0
    # via aiohttp
certifi==2025.11.12
    # via requests
charset-normalizer==3.4.4
    # via requests
filelock==3.20.1
    # via
    #   huggingface-hub
    #   torch
    #   transformers
frozenlist==1.8.0
    # via
    #   aiohttp
    #   aiosignal
fsspec==2025.12.0
    # via
    #   huggingface-hub
    #   lightning
    #   pytorch-lightning
    #   torch
hf-xet==1.2.0
    # via huggingface-hub
huggingface-hub==0

In [3]:
from sentence_transformers import SentenceTransformer

ImportError: huggingface-hub>=0.23.0,<1.0 is required for a normal functioning of this module, but found huggingface-hub==1.2.3.
Try: `pip install transformers -U` or `pip install -e '.[dev]'` if you're working with git main

# Run

In [3]:
from runtime.ultils import gdown_folder
gdown_folder()

Retrieving folder contents


Retrieving folder 1aoddGUNcsdJuD6KtSRigVnqLW4uqeHin .ipynb_checkpoints
Processing file 1pQevQ6Ors5DVuAXLxXiWkGh9LGi63DjE hallu_classes.csv
Processing file 1BWfYcKTw1lBcEpeINvcsL2sW36hsadYT hallu_data.csv


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1pQevQ6Ors5DVuAXLxXiWkGh9LGi63DjE
To: /kaggle/working/mlproduction/data/hallu_classes.csv
100%|██████████| 25.0/25.0 [00:00<00:00, 95.2kB/s]


Folder downloaded successfully.


Downloading...
From: https://drive.google.com/uc?id=1BWfYcKTw1lBcEpeINvcsL2sW36hsadYT
To: /kaggle/working/mlproduction/data/hallu_data.csv
100%|██████████| 10.7M/10.7M [00:00<00:00, 81.5MB/s]
Download completed


# What is the dog doing?


In [4]:
import dagshub
dagshub.init(repo_owner='ollamagena', repo_name='mlpro', mlflow=True)


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=08deda0c-dba2-430e-ae83-2993622be3c4&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=244189de78902645afdfa9c6a2e8ee15d6cd2db6c1c7cdfe761c516401a38e2c




Accessing as ollamagena

Initialized MLflow to track repo "ollamagena/mlpro"

Repository ollamagena/mlpro initialized!

# Run

In [7]:
import pandas as pd
from src.dataloaders.ultils import train_test_split_df, load_label_enocder

df = pd.read_csv("data/hallu_data.csv")
df.head()
le = load_label_enocder("data/hallu_classes.csv")
X_train, X_test, y_train, y_test = train_test_split_df(df, "label", le, 0.3)

In [28]:
from src.models.preprop_svm import Preprop, prep_remove_common_words_with_counting, agg_differential
from sklearn.svm import SVC

preprop = Preprop(prep_remove_common_words_with_counting, agg_differential)
svc = SVC()

ImportError: cannot import name 'cached_download' from 'huggingface_hub' (/usr/local/lib/python3.11/dist-packages/huggingface_hub/__init__.py)